In [1]:
import requests
import time
import math
import pandas as pd
import bisect
import datetime

In [6]:
tickers = {
    # 'Property & Construction': ['24CS', 'A5', 'ALL', 'ARIN', 'ARROW', 'BC', 'BLESS', 'BSM', 'BTW', 'CAZ', 'CHEWA', 'CPANEL', 'CRD', 'DHOUSE', 'DIMET', 'DPAINT', 'FLOYD', 'HYDRO', 'IND', 'JAK', 'K', 'KUN', 'META', 'PPS', 'PRI', 'PROS', 'PROUD', 'PSG', 'SENX', 'SK', 'SMART', 'SSS', 'STC', 'SVR', 'TAPAC', 'THANA', 'TIGER', 'TITLE', 'YONG'], 
    # 'Steel and Metal Products': ['2S', 'AMC', 'BSBM', 'CEN', 'CITY', 'CSP', 'GJS', 'GSTEEL', 'INOX', 'LHK', 'MCS', 'MILL', 'PAP', 'PERM', 'SAM', 'SMIT', 'SSSC', 'TGPRO', 'THE', 'TMT', 'TSTH', 'TWP', 'TYCN'], 
    # 'Automotive': ['3K-BAT', 'ACG', 'AH', 'CWT', 'EASON', 'GYT', 'HFT', 'IHL', 'INGRS', 'IRC', 'MGC', 'PCSGH', 'POLY', 'SAT', 'SPG', 'STANLY', 'TKT', 'TNPC', 'TRU', 'TSC'], 
    # 'Energy & Utilities': ['7UP', 'ACC', 'ACE', 'AGE', 'AI', 'AIE', 'AKR', 'BAFS', 'BANPU', 'BBGI', 'BCP', 'BCPG', 'BGRIM', 'BPP', 'BRRGIF', 'CKP', 'CV', 'DEMCO', 'EA', 'EASTW', 'EGATIF', 'EGCO', 'EP', 'ESSO', 'ETC', 'GPSC', 'GREEN', 'GULF', 'GUNKUL', 'IFEC', 'IRPC', 'JR', 'KBSPIF', 'LANNA', 'MDX', 'NOVA', 'OR', 'PCC', 'PRIME', 'PTG', 'PTT', 'PTTEP', 'QTC', 'RATCH', 'RPC', 'SCG', 'SCI', 'SCN', 'SGP', 'SKE', 'SOLAR', 'SPCG', 'SPRC', 'SSP', 'SUPER', 'SUPEREIF', 'SUSCO', 'TAE', 'TCC', 'TGE', 'TOP', 'TPIPP', 'TSE', 'TTW', 'UBE', 'WHAUP', 'WP'], 
    # 'Property Development': ['A', 'AKS', 'AMATA', 'AMATAV', 'ANAN', 'AP', 'APEX', 'ASW', 'AWC', 'BLAND', 'BRI', 'BROCK', 'CGD', 'CI', 'CMC', 'CPN', 'ESTAR', 'EVER', 'FPT', 'GLAND', 'J', 'JCK', 'KC', 'LALIN', 'LH', 'LPN', 'MBK', 'MJD', 'MK', 'NCH', 'NNCL', 'NOBLE', 'NUSA', 'NVD', 'ORI', 'PACE', 'PEACE', 'PF', 'PIN', 'PLAT', 'POLAR', 'PRECHA', 'PRIN', 'PSH', 'QH', 'RABBIT', 'RICHY', 'RML', 'ROJNA', 'S', 'SA', 'SAMCO', 'SC', 'SENA', 'SIRI', 'SPALI', 'UV', 'WHA', 'WIN'], 
    # 'Food & Beverage': ['AAI', 'APURE', 'ASIAN', 'BR', 'BRR', 'BTG', 'CBG', 'CFRESH', 'CH', 'CHOTI', 'CM', 'CPF', 'CPI', 'F&D', 'GLOCON', 'HTC', 'ICHI', 'ITC', 'JDF', 'KBS', 'KSL', 'KTIS', 'LST', 'M', 'MALEE', 'MINT', 'NRF', 'NSL', 'OISHI', 'OSP', 'PB', 'PLUS', 'PM', 'PQS', 'PRG', 'RBF', 'SAPPE', 'SAUCE', 'SFP', 'SNNP', 'SNP', 'SORKON', 'SSC', 'SSF', 'SST', 'SUN', 'TC', 'TFG', 'TFMAMA', 'TIPCO', 'TKN', 'TU', 'TVO', 'W', 'ZEN'], 
    # 'Transportation & Logistics': ['AAV', 'AOT', 'ASIMAR', 'B', 'BA', 'BEM', 'BIOTEC', 'BTS', 'BTSGIF', 'DMT', 'III', 'KEX', 'KIAT', 'KWC', 'MENA', 'NOK', 'NYT', 'PORT', 'PRM', 'PSL', 'RCL', 'SJWD', 'TFFIF', 'THAI', 'TSTE', 'TTA', 'WICE'], 'Resources': ['ABM', 'GTV', 'PSTC', 'PTC', 'SAAM', 'SEAOIL', 'SR', 'STOWER', 'TAKUNI', 'TPCH', 'TRT', 'UMS'], 
    # 'Financials': ['ACAP', 'AF', 'AIRA', 'ASN', 'BROOK', 'GCAP', 'KCC', 'LIT', 'MITSIB', 'SGF', 'TQR'], 
    # 'Industrial': ['ADB', 'BM', 'CHO', 'CHOW', 'CIG', 'COLOR', 'CPR', 'FPI', 'GTB', 'KCM', 'KJL', 'KUMWEL', 'KWM', 'MBAX', 'MGT', 'MTW', 'NDR', 'PACO', 'PDG', 'PIMO', 'PPM', 'PRAPAT', 'RWI', 'SAF', 'SALEE', 'SANKO', 'SELIC', 'SFT', 'STP', 'SWC', 'TMC', 'TMI', 'TMW', 'TPLAS', 'TRV', 'UBIS', 'UEC', 'UKEM', 'UREKA', 'YUASA', 'ZIGA'], 
    # 'Services': ['ADD', 'AKP', 'AMA', 'AMARC', 'ARIP', 'ATP30', 'AUCT', 'BIS', 'BOL', 'CEYE', 'CHIC', 'CMO', 'D', 'DEXON', 'DV8', 'ETE', 'FSMART', 'FVC', 'GLORY', 'GSC', 'HARN', 'HEALTH', 'HL', 'IMH', 'KGEN', 'KK', 'KLINIQ', 'KOOL', 'KTMS', 'LDC', 'LEO', 'MASTER', 'MEB', 'MORE', 'MVP', 'NCL', 'NEWS', 'OTO', 'PHOL', 'PICO', 'PLT', 'QLT', 'RP', 'SE', 'SLM', 'SONIC', 'SPA', 'THMUI', 'TNDT', 'TNH', 'TNP', 'TSF', 'TURTLE', 'TVDH', 'TVT', 'UBA', 'VL', 'WINNER', 'YGG'], 
    # 'Information & Communication Technology': ['ADVANC', 'AIT', 'ALT', 'AMR', 'BLISS', 'DIF', 'DTCENT', 'FORTH', 'GABLE', 'HUMAN', 'ILINK', 'INET', 'INSET', 'INTUCH', 'ITEL', 'JAS', 'JASIF', 'JMART', 'JTS', 'MFEC', 'MSC', 'PT', 'SAMART', 'SAMTEL', 'SDC', 'SIS', 'SKY', 'SVOA', 'SYMC', 'SYNEX', 'THCOM', 'TKC', 'TRUE', 'TWZ'], 
    # 'Finance & Securities': ['AEONTS', 'AMANAH', 'ASAP', 'ASK', 'ASP', 'BAM', 'BYD', 'CGH', 'CHASE', 'CHAYO', 'ECL', 'FNS', 'FSS', 'GBX', 'GL', 'HENG', 'IFS', 'JMT', 'KCAR', 'KGI', 'KTC', 'MFC', 'MICRO', 'ML', 'MST', 'MTC', 'NCAP', 'PL', 'S11', 'SAK', 'SAWAD', 'SCAP', 'SGC', 'SM', 'TH', 'THANI', 'TIDLOR', 'TK', 'TNITY', 'UOBKH', 'XPG'], 
    # 'Fashion': ['AFC', 'AURA', 'BTNC', 'CPH', 'CPL', 'NC', 'PAF', 'PDJ', 'PG', 'SABINA', 'SAWANG', 'SUC', 'TNL', 'TR', 'TTI', 'TTT', 'UPF', 'WACOAL', 'WFX'], 
    # 'Health Care Services': ['AHC', 'BCH', 'BDMS', 'BH', 'CHG', 'CMR', 'EKH', 'KDH', 'LPH', 'M-CHAI', 'NEW', 'NTV', 'PR9', 'PRINC', 'RAM', 'RJH', 'RPH', 'SKR', 'THG', 'VIBHA', 'VIH', 'WPH'], 
    # 'Property Fund & REITs': ['AIMCG', 'AIMIRT', 'ALLY', 'AMATAR', 'BAREIT', 'BKKCP', 'BOFFICE', 'B-WORK', 'CPNCG', 'CPNREIT', 'CPTGF', 'CTARAF', 'DREIT', 'ERWPF', 'FTREIT', 'FUTUREPF', 'GAHREIT', 'GROREIT', 'GVREIT', 'HPF', 'HYDROGEN', 'IMPACT', 'INETREIT', 'KPNPF', 'KTBSTMR', 'LHHOTEL', 'LHPF', 'LHSC', 'LPF', 'LUXF', 'M-II', 'MIPF', 'MIT', 'MJLF', 'MNIT', 'MNIT2', 'MNRF', 'M-PAT', 'M-STOR', 'POPF', 'PPF', 'PROSPECT', 'QHHR', 'QHOP', 'QHPF', 'SHREIT', 'SIRIP', 'SPRIME', 'SRIPANWA', 'SSPF', 'SSTRT', 'TIF1', 'TLHPF', 'TNPF', 'TPRIME', 'TTLPF', 'TU-PF', 'URBNPF', 'WHABT', 'WHAIR', 'WHART'], 
    # 'Packaging': ['AJ', 'ALUCON', 'BGC', 'CSC', 'NEP', 'PJW', 'PTL', 'SCGP', 'SFLEX', 'SITHAI', 'SLP', 'SMPC', 'SPACK', 'TCOAT', 'TFI', 'THIP', 'TMD', 'TOPP', 'TPAC', 'TPBI', 'TPP'], 
    # 'Home & Office Products': ['AJA', 'DTCI', 'FANCY', 'FTI', 'KYE', 'L&E', 'MODERN', 'OGC', 'ROCK', 'SBNEXT', 'SIAM', 'TCMC'], 
    # 'Industrial Materials & Machinery': ['ALLA', 'ASEFA', 'CPT', 'CRANE', 'CTW', 'FMT', 'HTECH', 'KKC', 'PK', 'SNC', 'STARK', 'TCJ', 'TPCS', 'VARO'], 
    # 'Consumer Products': ['ALPHAX', 'BGT', 'DOD', 'ECF', 'EFORL', 'HPT', 'IP', 'ITTHI', 'JSP', 'JUBILE', 'MOONG', 'NPK', 'SMD', 'TM', 'WARRIX', 'WINMED'], 
    # 'Media & Publishing': ['AMARIN', 'AQUA', 'AS', 'BEC', 'FE', 'GPI', 'GRAMMY', 'JKN', 'MACO', 'MAJOR', 'MATCH', 'MATI', 'MCOT', 'MONO', 'MPIC', 'NATION', 'ONEE', 'PLANB', 'POST', 'PRAKIT', 'PTECH', 'SE-ED', 'TKS', 'VGI', 'WAVE', 'WORK'], 
    # 'Personal Products & Pharmaceuticals': ['APCO', 'BIZ', 'DDD', 'JCT', 'KISS', 'NV', 'OCC', 'S&J', 'STGT', 'STHAI', 'TNR', 'TOG'], 
    # 'Construction Services': ['APCS', 'BJCHI', 'BKD', 'CIVIL', 'CK', 'CNT', 'EMC', 'ITD', 'NWR', 'PLE', 'PREB', 'PYLON', 'RT', 'SEAFCO', 'SQ', 'SRICHA', 'STEC', 'STI', 'STPI', 'SYNTEC', 'TEAMG', 'TEKA', 'TPOLY', 'TRC', 'TRITN', 'TTCL', 'UNIQ', 'WGE'], 
    # 'Technology': ['APP', 'BBIK', 'BE8', 'BVG', 'COMAN', 'DITTO', 'ICN', 'IIG', 'IRCP', 'ITNS', 'NETBAY', 'PLANET', 'PROEN', 'READY', 'SECURE', 'SICT', 'SIMAT', 'SPVI', 'TPS', 'VCOM'], 'Tourism & Leisure': ['ASIA', 'BEYOND', 'CENTEL', 'CSR', 'DUSIT', 'ERW', 'GRAND', 'LRH', 'MANRIN', 'OHTL', 'ROH', 'SHANG', 'SHR', 'VRANDA'], 
    # 'Agro & Food Industry': ['AU', 'JCKH', 'KASET', 'MUD', 'NTSC', 'TACC', 'TMILL', 'XO'], 'Insurance': ['AYUD', 'BKI', 'BLA', 'BUI', 'CHARAN', 'INSURE', 'KWI', 'MTI', 'NKI', 'SMK', 'TGH', 'THRE', 'THREL', 'TIPH', 'TLI', 'TQM', 'TSI', 'TVI'], 
    # 'Commerce': ['B52', 'BEAUTY', 'BIG', 'BJC', 'COM7', 'CPALL', 'CPW', 'CRC', 'CSS', 'DOHOME', 'FN', 'FTE', 'GLOBAL', 'HMPRO', 'ICC', 'ILM', 'IT', 'KAMART', 'LOXLEY', 'MAKRO', 'MC', 'MEGA', 'MIDA', 'MOSHI', 'RS', 'RSP', 'SABUY', 'SCM', 'SINGER', 'SPC', 'SPI', 'SVT'], 'Banking': ['BAY', 'BBL', 'CIMBT', 'KBANK', 'KKP', 'KTB', 'LHFG', 'SCB', 'TCAP', 'TISCO', 'TTB'], 
    # 'Petrochemicals & Chemicals': ['BCT', 'CMAN', 'GC', 'GGC', 'GIFT', 'IVL', 'NFC', 'PATO', 'PMTA', 'PTTGC', 'SUTHA', 'TCCC', 'TPA', 'UAC', 'UP'], 
    'Professional Services': ['BWG', 'GENCO', 'PRO', 'PRTR', 'SISB', 'SO'], 
    # 'Electronic Components': ['CCET', 'DELTA', 'HANA', 'KCE', 'METCO', 'NEX', 'SMT', 'SVI', 'TEAM'], 
    # 'Construction Materials': ['CCP', 'COTTO', 'DCC', 'DCON', 'DRT', 'EPG', 'GEL', 'PPP', 'Q-CON', 'SCC', 'SCCC', 'SCP', 'SKN', 'STECH', 'TASCO', 'TOA', 'TPIPL', 'UMI', 'VNG', 'WIIK'], 
    # 'Agribusiness': ['EE', 'GFPT', 'LEE', 'MAX', 'NER', 'PPPM', 'STA', 'TEGH', 'TFM', 'TRUBB', 'TWPC', 'UPOIC', 'UVAN', 'VPO'], 
    # 'Companies Under Rehabilitation': ['POMPUI', 'SAFARI', 'WORLD'], 
    # 'Mining': ['THL'], 
    # 'Paper & Printing Materials': ['UTP']
}

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

cur_ts = math.floor(time.time())

In [8]:
tab = []

for sector in tickers:
    for ticker in tickers[sector]:
        try:
            
    #import data
            FS_res = requests.get(f'https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/{ticker}.BK?lang=en-US&region=US&symbol={ticker}.BK&padTimeSeries=true&type=annualWorkingCapital,annualTotalAssets,annualRetainedEarnings,annualShareIssued,annualTotalDebt,annualTotalRevenue,annualEBIT&merge=false&period1=850262400&period2={cur_ts}&corsDomain=finance.yahoo.com', headers=headers)
            price_res = requests.get(f'https://query1.finance.yahoo.com/v8/finance/chart/{ticker}.BK?formatted=true&crumb=3PtFND5i2mA&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=850262400&period2={cur_ts}&events=capitalGain|div|split&useYfid=true&corsDomain=finance.yahoo.com', headers=headers)

            FS = FS_res.json()
            PRICE = price_res.json()
                    
    #get number 
            items = {}
            for item in FS['timeseries']['result']:
                items[item['meta']['type'][0]] = {}
                items['price'] = {}
                for data in item[item['meta']['type'][0]]:
                    data_ts = time.mktime(datetime.datetime.strptime(data['asOfDate'], '%Y-%m-%d').timetuple())
                    items[item['meta']['type'][0]][int(data_ts)] = data['reportedValue']['raw']
                    
                    # price
                    r_idx = bisect.bisect_left(PRICE['chart']['result'][0]['timestamp'], data_ts) - 1
                    price = PRICE['chart']['result'][0]['indicators']['quote'][0]['close'][r_idx]
                    items['price'][int(data_ts)] = price
            
            # print(ticker)
        #calculate z score    
            for yr_ts in items['annualWorkingCapital']:
                try:
                    A = items['annualWorkingCapital'][yr_ts]/items['annualTotalAssets'][yr_ts]
                    B = items['annualRetainedEarnings'][yr_ts]/items['annualTotalAssets'][yr_ts]
                    C = items['annualEBIT'][yr_ts]/items['annualTotalAssets'][yr_ts]
                    D = (items['price'][yr_ts] * items['annualShareIssued'][yr_ts])/items['annualTotalDebt'][yr_ts]
                    E = items['annualTotalRevenue'][yr_ts]/items['annualTotalAssets'][yr_ts]

                    # print(datetime.datetime.fromtimestamp(yr_ts).strftime('%Y-%m-%d'), 1.2*A + 1.4*B + 3.3*C + 0.6*D + 1.0*E)

                    data = {
                        'Ticker': ticker,
                        'Sector': sector,
                        'Reported_date': datetime.datetime.fromtimestamp(yr_ts).strftime('%Y-%m-%d'),
                        'Working_cap': items['annualWorkingCapital'][yr_ts],
                        'Total_asset': items['annualTotalAssets'][yr_ts],
                        'Retained_earning': items['annualRetainedEarnings'][yr_ts],
                        'EBIT': items['annualEBIT'][yr_ts],
                        'Market_cap': (items['price'][yr_ts] * items['annualShareIssued'][yr_ts]),
                        'Total_liabilities': items['annualTotalDebt'][yr_ts],
                        'Sales': items['annualTotalRevenue'][yr_ts],
                        'z_score': 1.2*A + 1.4*B + 3.3*C + 0.6*D + 1.0*E
                    }

                    tab.append(data)
                except:
                    print(datetime.datetime.fromtimestamp(yr_ts).strftime('%Y-%m-%d') + 'error')
        
            # tab.append(data)
            
        except:
            tab.append({
                        'Ticker': ticker,
                        'Sector': sector,
                        'Reported_date': '-',
                        'Working_cap': '-',
                        'Total_asset': '-',
                        'Retained_earning': '-',
                        'EBIT': '-',
                        'Market_cap': '-',
                        'Total_liabilities': '-',
                        'Sales': '-',
                        'z_score': 'error'
                    })
            print(ticker + ' error')

    dataframe = pd.DataFrame(tab)
    # dataframe.to_excel("zscore_set.xlsx")
    display(dataframe)

PRO error
PRTR error


,Ticker,Sector,Reported_date,Working_cap,Total_asset,Retained_earning,EBIT,Market_cap,Total_liabilities,Sales,z_score
0,BWG,Professional Services,2019-12-31,-199453713.0,7356427906.0,1030568888.0,130232424.0,2337592583.818901,3147281698.0,1879785877.0,0.923183
1,BWG,Professional Services,2020-12-31,971131612.0,8846770104.0,852254464.0,104031732.0,2034048300.588155,2987400975.0,1915319461.0,0.930426
2,BWG,Professional Services,2021-12-31,427606688.0,8321560379.0,687113220.0,115108990.0,4232540609.552139,1510282665.0,2182692201.0,2.166692
3,BWG,Professional Services,2022-12-31,1017514060.0,8581814972.0,548353933.0,85593435.0,3322316883.805704,2610947996.0,2503200004.0,1.319809
4,GENCO,Professional Services,2019-12-31,518015308.0,1566958526.0,44310422.0,-30004448.0,617263707.12883,68046570.0,282704922.0,5.996237
5,GENCO,Professional Services,2020-12-31,445034183.0,1527528377.0,-15300005.0,-40243522.0,549925846.953064,102877686.0,256800482.0,3.624024
6,GENCO,Professional Services,2021-12-31,355685792.0,1508227871.0,-9682833.0,10135520.0,1043736799.277298,71618183.0,210383129.0,9.179852
7,GENCO,Professional Services,2022-12-31,348642431.0,1536766785.0,-15437593.0,-15263895.0,673378601.757661,57440614.0,253224426.0,7.424001
8,PRO,Professional Services,-,-,-,-,-,-,-,-,error
9,PRTR,Professional Services,-,-,-,-,-,-,-,-,error
